In [2]:
# Import Libraries
import pandas as pd
from nltk.corpus import stopwords
import requests
import pyterrier as pt
from Preprocessing.Preprocess import split_by_language,compute_similarity,convert2json
if not pt.started():
    pt.init()

[nltk_data] Error loading punkt: <urlopen error [Errno 11002]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading punkt_tab: <urlopen error [Errno 11002]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading stopwords: <urlopen error [Errno 11002]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading stopwords: <urlopen error [Errno 11002]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading punkt: <urlopen error [Errno 11002]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading punkt_tab: <urlopen error [Errno 11002]
[nltk_data]     getaddrinfo failed>
C:\Users\Mohammad Kher\AppData\Local\Temp\ipykernel_7192\1770673465.py:7: DeprecationWarning: Call to deprecated function (or staticmethod) started. (use pt.java.started() instead) -- Deprecated since version 0.11.0.
  if not pt.started():
c:\Users\Mohammad Kher\Desktop\University Projects\Legal-Assistance-AI\venv\Lib\site-packages\pyterrier\java\mavenresolver.py:156: UserWarning: Attempted to

In [4]:

### Load The Data From The APIs

# Get Lawyers From Lawyers API
lawyer_response = requests.get("http://osamanaser2003-21041.portmap.host:21041/api/v1/ai/lawyers")
print(f"status code : {lawyer_response.status_code}")


# Get Rates From Rates API 
rate_response = requests.get("http://osamanaser2003-21041.portmap.host:21041/api/v1/ai/ratings")
print(f"status code : {rate_response.status_code}")


# Get Agencies From Agencies API
agency_response = requests.get("http://osamanaser2003-21041.portmap.host:21041/api/v1/ai/agencies")
print(f"status code : {agency_response.status_code}")


# Get Issues From Issues API
issue_response = requests.get("http://osamanaser2003-21041.portmap.host:21041/api/v1/ai/issues")
print(f"status code : {issue_response.status_code}")

status code : 200
status code : 200
status code : 200
status code : 200


In [11]:
### Lawyers Data Perprocessing
lawyers_response = lawyer_response.json()
lawyers_pere = pd.DataFrame(lawyers_response['data'])
lawyers = lawyers_pere.rename(columns={
    "id":"lawyer_id"
}).drop(["email","union_number","affiliation_date","phone","avatar","credit_cart","reservation_profit","conversation_profit","role"],axis=1)
lawyers['years_of_experience'] = lawyers['years_of_experience'].apply(lambda x :f"{x}year")
lawyers.head()

,lawyer_id,name,address,birthdate,birth_place,national_number,union_branch,years_of_experience,description,gender,rank,rates,specializations,agencies,issues
0,115,ميساء المقبل,54 شارع يونس الشهري\nالهفوف,1988-12-05,Boganberg,None,Lake Rethamouth,38year,"Hatter, 'I cut some more bread-and-butter--' '...",female,0.0,[],"[{'id': 6, 'name': 'Commercial'}]",[],[]
1,116,السيدة بشرى الشهيل,39901 طريق رامان سقا عمارة رقم 29\nالطائف,2011-04-01,Port Javonside,None,Bofurt,36year,Alice)--'and perhaps you haven't found it made...,male,4.0,"[{'id': 76, 'user_id': 114, 'user_name': 'داوو...","[{'id': 2, 'name': 'Civil'}]",[],[]
2,117,رضوان اوس المقبل,4241 طريق عبلة القحطاني شقة رقم. 19\nالطائف,1972-02-23,Lake Erynborough,None,Schulisttown,12year,Alice. One of the right-hand bit to try the pa...,female,0.0,[],"[{'id': 2, 'name': 'Civil'}]",[],[]
3,118,وفاء الحميد,27698 ممر ليدا الصقير\nوادي الدواسر,2001-01-04,Krishaven,None,Hahnbury,38year,"Alice felt a very difficult question. However,...",male,0.0,[],"[{'id': 4, 'name': 'Military'}]",[],[]
4,119,الآنسة وسام باشا,93891 شارع عبدالرؤوف السيف\nسيهات,1984-03-09,North Cordell,None,Lake Roxaneshire,10year,Alice indignantly. 'Ah! then yours wasn't a re...,male,0.0,[],"[{'id': 7, 'name': 'Arbitration'}]",[],[]


In [12]:
rates_response = rate_response.json()
rates = pd.DataFrame(rates_response["data"])
rates.head()

,id,user_id,user_name,user_avatar,lawyer_id,rating,review,timestamp
0,1,114,داوود مهران هوساوي,https://via.placeholder.com/640x480.png/0000aa...,225,2,أتممها لشروق النور الأعلى عليها استعدادً، وان ...,2025-08-24 14:14:54
1,3,56,يزن برماوي,https://via.placeholder.com/640x480.png/00ffff...,308,1,بهائه، ولم يعرض عنه حتى وافته منيته، وهذا على ...,2025-08-24 14:14:54
2,4,80,رنده السمير,https://via.placeholder.com/640x480.png/00aa44...,160,1,بما هو جسم، مركب على الحقيقة من معنين: أحدهما ...,2025-08-24 14:14:54
3,5,33,ميران عثمان ضرار المطرفي,https://via.placeholder.com/640x480.png/007700...,299,5,و الاموات، ولم يزل أسال يرغب إليه ويستعطفه. وق...,2025-08-24 14:14:54
4,6,65,السيدة غفران الشهيل,https://via.placeholder.com/640x480.png/0088ee...,260,1,الذات صورة الشمس التي تظهر في ماء مترجرج، وقد ...,2025-08-24 14:14:54


In [13]:
### Rates Data Preproceesing
rates_response = rate_response.json()
rates = pd.DataFrame(rates_response["data"])
rates.dropna(inplace=True)
rates.drop(["id"],axis=1,inplace=True)
rates['rating'] = rates['rating'].apply(lambda x :f"{x}star")
rates.head()

,user_id,user_name,user_avatar,lawyer_id,rating,review,timestamp
0,114,داوود مهران هوساوي,https://via.placeholder.com/640x480.png/0000aa...,225,2star,أتممها لشروق النور الأعلى عليها استعدادً، وان ...,2025-08-24 14:14:54
1,56,يزن برماوي,https://via.placeholder.com/640x480.png/00ffff...,308,1star,بهائه، ولم يعرض عنه حتى وافته منيته، وهذا على ...,2025-08-24 14:14:54
2,80,رنده السمير,https://via.placeholder.com/640x480.png/00aa44...,160,1star,بما هو جسم، مركب على الحقيقة من معنين: أحدهما ...,2025-08-24 14:14:54
3,33,ميران عثمان ضرار المطرفي,https://via.placeholder.com/640x480.png/007700...,299,5star,و الاموات، ولم يزل أسال يرغب إليه ويستعطفه. وق...,2025-08-24 14:14:54
4,65,السيدة غفران الشهيل,https://via.placeholder.com/640x480.png/0088ee...,260,1star,الذات صورة الشمس التي تظهر في ماء مترجرج، وقد ...,2025-08-24 14:14:54


In [14]:
### Agencies Data Perprcessing
agencies_response = agency_response.json()
agencies = pd.DataFrame(agencies_response["data"]).rename(columns={
    "id":"agency_id"
})
agencies = agencies[["agency_id","lawyer_id"]]
agencies.head()

,agency_id,lawyer_id
0,6,197
1,17,183
2,22,312
3,23,307
4,26,238


In [15]:
### Issues Data Preprocessing
issues_response = issue_response.json()
issues = pd.DataFrame(issues_response["data"]).drop(["base_number","record_number","id","start_date","end_date","status"],axis=1)
issues.head()

,agency_id,agency_sequential,agency_record,court_id,court_name,court_room_id,court_room_name,estimated_cost,is_active,is_archived,success_rate,agency
0,4,44581766,37683312,1,Cassation,1,Cassation,10884613,No,Yes,True,"{'id': 4, 'sequential_number': '44581766', 're..."
1,31,None,None,3,Criminal,9,Penalty Beginning,5776410,Yes,No,None,"{'id': 31, 'sequential_number': None, 'record_..."
2,31,None,None,3,Criminal,9,Penalty Beginning,19037115,No,Yes,False,"{'id': 31, 'sequential_number': None, 'record_..."
3,85,86518562,29039359,3,Criminal,11,Criminals,29815330,No,Yes,False,"{'id': 85, 'sequential_number': '86518562', 'r..."
4,51,None,None,2,Civil,6,Banking,6343424,No,Yes,True,"{'id': 51, 'sequential_number': None, 'record_..."


In [16]:
### Merge Lawyers With Rates
lawyers_with_rates = lawyers.merge(rates,on="lawyer_id")
lawyers_with_rates.head()


,lawyer_id,name,address,birthdate,birth_place,national_number,union_branch,years_of_experience,description,gender,...,rates,specializations,agencies,issues,user_id,user_name,user_avatar,rating,review,timestamp
0,116,السيدة بشرى الشهيل,39901 طريق رامان سقا عمارة رقم 29\nالطائف,2011-04-01,Port Javonside,None,Bofurt,36year,Alice)--'and perhaps you haven't found it made...,male,...,"[{'id': 76, 'user_id': 114, 'user_name': 'داوو...","[{'id': 2, 'name': 'Civil'}]",[],[],114,داوود مهران هوساوي,https://via.placeholder.com/640x480.png/0000aa...,4star,لنفسه شبيهاً فيهم. وكان أيضاً إذا رجع إلى ذاته...,2025-08-24 14:15:02
1,120,رسلان السالم,88 شارع مالك السمير شقة رقم. 15\nسيهات,1999-03-27,North Kris,None,West Alyssonville,18year,"Mouse. 'Of course,' the Gryphon in an agony of...",female,...,"[{'id': 15, 'user_id': 91, 'user_name': 'رهام ...","[{'id': 2, 'name': 'Civil'}]",[],[],91,رهام الصامل,https://via.placeholder.com/640x480.png/0066cc...,3star,لها، وهي معلومة له، سواء كانت من صورة الإنسان ...,2025-08-24 14:14:54
2,121,سيف الزامل,5122 شارع سيرين هوساوي\nبيشة,2024-01-24,Kirstenfort,None,Liatown,21year,I will just explain to you never tasted an egg...,female,...,"[{'id': 9, 'user_id': 34, 'user_name': 'إنعام ...","[{'id': 1, 'name': 'Legal'}]",[],[],34,إنعام الجهني,https://via.placeholder.com/640x480.png/0000cc...,1star,ولا بهاء، ولا جمال إلا صادر من جهته، وفائض من ...,2025-08-24 14:14:54
3,123,ميس الزامل,90667 طريق وسن الخضيري\nالثقبة,1978-09-30,Lake Dorris,None,Lednerside,21year,"He looked at poor Alice, 'to speak to this las...",male,...,"[{'id': 33, 'user_id': 48, 'user_name': 'أنوار...","[{'id': 6, 'name': 'Commercial'}]",[],[],48,أنوار سعود عبدالعزيز الخالدي,https://via.placeholder.com/640x480.png/0033dd...,3star,إلى غير نهاية، وان وصل الحجر إلى غير نهاية ولا...,2025-08-24 14:14:54
4,123,ميس الزامل,90667 طريق وسن الخضيري\nالثقبة,1978-09-30,Lake Dorris,None,Lednerside,21year,"He looked at poor Alice, 'to speak to this las...",male,...,"[{'id': 33, 'user_id': 48, 'user_name': 'أنوار...","[{'id': 6, 'name': 'Commercial'}]",[],[],65,السيدة غفران الشهيل,https://via.placeholder.com/640x480.png/0088ee...,5star,كذلك، فرأى أن يكون في سبب نجاته. فرآها كلها إن...,2025-08-24 14:15:02


In [17]:
### Merge Lawyers_with_rates with Agencies
lawyers_with_rates= pd.merge(agencies,lawyers_with_rates,on=["lawyer_id"],how="inner")
lawyers_with_rates.head()

,agency_id,lawyer_id,name,address,birthdate,birth_place,national_number,union_branch,years_of_experience,description,...,rates,specializations,agencies,issues,user_id,user_name,user_avatar,rating,review,timestamp
0,6,197,كرم الأحمري,6901 شارع هيا الحميد عمارة رقم 35\nبيشة,1988-04-21,Port Marcelostad,None,New Faustoview,15year,"Mock Turtle, 'they--you've seen them, of cours...",...,"[{'id': 44, 'user_id': 23, 'user_name': 'شهد ي...","[{'id': 9, 'name': 'International Disputes'}]",[],[],23,شهد يعرب الصامل,https://via.placeholder.com/640x480.png/00bb11...,5star,الحيوان؛ وما فيه من نقص الفطرة والأعراض عن آمر...,2025-08-24 14:14:54
1,6,197,كرم الأحمري,6901 شارع هيا الحميد عمارة رقم 35\nبيشة,1988-04-21,Port Marcelostad,None,New Faustoview,15year,"Mock Turtle, 'they--you've seen them, of cours...",...,"[{'id': 44, 'user_id': 23, 'user_name': 'شهد ي...","[{'id': 9, 'name': 'International Disputes'}]",[],[],59,السيدة روند السهلي,https://via.placeholder.com/640x480.png/00dd88...,1star,الضعفاء الذين اطرحوا تقليد الأنبياء صلوات الله...,2025-08-24 14:15:02
2,23,307,آية السالم,21 ممر حصة الجريد شقة رقم. 48\nالطائف,1970-07-23,Weissnatbury,None,New Gileschester,37year,"They had not gone far before they saw her, the...",...,"[{'id': 22, 'user_id': 45, 'user_name': 'مهدي ...","[{'id': 1, 'name': 'Legal'}]",[],[],45,مهدي السيف,https://via.placeholder.com/640x480.png/0000ff...,1star,ثارت عنده من هذا التشبه لاستدامة هذا الروح الح...,2025-08-24 14:14:54
3,23,307,آية السالم,21 ممر حصة الجريد شقة رقم. 48\nالطائف,1970-07-23,Weissnatbury,None,New Gileschester,37year,"They had not gone far before they saw her, the...",...,"[{'id': 22, 'user_id': 45, 'user_name': 'مهدي ...","[{'id': 1, 'name': 'Legal'}]",[],[],38,قارس المنيف,https://via.placeholder.com/640x480.png/00ee77...,5star,مركباً تحمله إلى تلك الجزيرة، جزيرة عظيمة متسع...,2025-08-24 14:15:02
4,26,238,مأمون واصف عبداللطيف العنزي,59397 طريق برهان الفرحان عمارة رقم 86\nالرياض,1985-01-19,Brendonberg,None,Skilesmouth,26year,HE was.' 'I never was so long since she had re...,...,"[{'id': 10, 'user_id': 15, 'user_name': 'المهن...","[{'id': 9, 'name': 'International Disputes'}]",[],[],15,المهندسة داليا الحسين,https://via.placeholder.com/640x480.png/00dd33...,5star,يجب عليه من حال المشاهدة، إلا بعد التمرن والاع...,2025-08-24 14:14:54


In [18]:
### Merge Them With Issues
lawyers_with_rates = pd.merge(issues,lawyers_with_rates,on="agency_id",how="inner")
lawyers_with_rates.head()

,agency_id,agency_sequential,agency_record,court_id,court_name,court_room_id,court_room_name,estimated_cost,is_active,is_archived,...,rates,specializations,agencies,issues,user_id,user_name,user_avatar,rating,review,timestamp
0,4,44581766,37683312,1,Cassation,1,Cassation,10884613,No,Yes,...,"[{'id': 51, 'user_id': 19, 'user_name': 'فهد ا...","[{'id': 1, 'name': 'Legal'}]",[],[],19,فهد الخالدي,https://via.placeholder.com/640x480.png/00ddee...,4star,البرهان على خلافه. وذلك أنه كان ينتقل إلى جميع...,2025-08-24 14:14:54
1,85,86518562,29039359,3,Criminal,11,Criminals,29815330,No,Yes,...,"[{'id': 132, 'user_id': 41, 'user_name': 'بدور...","[{'id': 1, 'name': 'Legal'}]",[],[],41,بدور وحيد عبدالرحمن الشيباني,https://via.placeholder.com/640x480.png/005555...,2star,إن يقال انها كثيرة، ولا واحدة، لان الكثرة انما...,2025-08-24 14:15:02
2,4,44581766,37683312,3,Criminal,11,Criminals,11168611,No,Yes,...,"[{'id': 51, 'user_id': 19, 'user_name': 'فهد ا...","[{'id': 1, 'name': 'Legal'}]",[],[],19,فهد الخالدي,https://via.placeholder.com/640x480.png/00ddee...,4star,البرهان على خلافه. وذلك أنه كان ينتقل إلى جميع...,2025-08-24 14:14:54
3,49,24157660,65579015,2,Civil,6,Banking,26919441,No,Yes,...,"[{'id': 3, 'user_id': 56, 'user_name': 'يزن بر...","[{'id': 3, 'name': 'Criminal'}]",[],[],56,يزن برماوي,https://via.placeholder.com/640x480.png/00ffff...,1star,بهائه، ولم يعرض عنه حتى وافته منيته، وهذا على ...,2025-08-24 14:14:54
4,49,24157660,65579015,2,Civil,6,Banking,26919441,No,Yes,...,"[{'id': 3, 'user_id': 56, 'user_name': 'يزن بر...","[{'id': 3, 'name': 'Criminal'}]",[],[],107,رغيد عزت أوس الحسين,https://via.placeholder.com/640x480.png/00aaee...,5star,الكون والفساد ومنها تتركب الأشياء ذوات الصور ا...,2025-08-24 14:15:02


In [19]:
lawyers_with_rates.head()

,agency_id,agency_sequential,agency_record,court_id,court_name,court_room_id,court_room_name,estimated_cost,is_active,is_archived,...,rates,specializations,agencies,issues,user_id,user_name,user_avatar,rating,review,timestamp
0,4,44581766,37683312,1,Cassation,1,Cassation,10884613,No,Yes,...,"[{'id': 51, 'user_id': 19, 'user_name': 'فهد ا...","[{'id': 1, 'name': 'Legal'}]",[],[],19,فهد الخالدي,https://via.placeholder.com/640x480.png/00ddee...,4star,البرهان على خلافه. وذلك أنه كان ينتقل إلى جميع...,2025-08-24 14:14:54
1,85,86518562,29039359,3,Criminal,11,Criminals,29815330,No,Yes,...,"[{'id': 132, 'user_id': 41, 'user_name': 'بدور...","[{'id': 1, 'name': 'Legal'}]",[],[],41,بدور وحيد عبدالرحمن الشيباني,https://via.placeholder.com/640x480.png/005555...,2star,إن يقال انها كثيرة، ولا واحدة، لان الكثرة انما...,2025-08-24 14:15:02
2,4,44581766,37683312,3,Criminal,11,Criminals,11168611,No,Yes,...,"[{'id': 51, 'user_id': 19, 'user_name': 'فهد ا...","[{'id': 1, 'name': 'Legal'}]",[],[],19,فهد الخالدي,https://via.placeholder.com/640x480.png/00ddee...,4star,البرهان على خلافه. وذلك أنه كان ينتقل إلى جميع...,2025-08-24 14:14:54
3,49,24157660,65579015,2,Civil,6,Banking,26919441,No,Yes,...,"[{'id': 3, 'user_id': 56, 'user_name': 'يزن بر...","[{'id': 3, 'name': 'Criminal'}]",[],[],56,يزن برماوي,https://via.placeholder.com/640x480.png/00ffff...,1star,بهائه، ولم يعرض عنه حتى وافته منيته، وهذا على ...,2025-08-24 14:14:54
4,49,24157660,65579015,2,Civil,6,Banking,26919441,No,Yes,...,"[{'id': 3, 'user_id': 56, 'user_name': 'يزن بر...","[{'id': 3, 'name': 'Criminal'}]",[],[],107,رغيد عزت أوس الحسين,https://via.placeholder.com/640x480.png/00aaee...,5star,الكون والفساد ومنها تتركب الأشياء ذوات الصور ا...,2025-08-24 14:15:02


In [20]:
### Add Data To Text Column
lawyers_with_rates["Text"] = lawyers_with_rates[["court_name",'address','union_branch',"review"]].astype(str).agg(" ".join,axis=1)
lawyers_with_rates.head()

,agency_id,agency_sequential,agency_record,court_id,court_name,court_room_id,court_room_name,estimated_cost,is_active,is_archived,...,specializations,agencies,issues,user_id,user_name,user_avatar,rating,review,timestamp,Text
0,4,44581766,37683312,1,Cassation,1,Cassation,10884613,No,Yes,...,"[{'id': 1, 'name': 'Legal'}]",[],[],19,فهد الخالدي,https://via.placeholder.com/640x480.png/00ddee...,4star,البرهان على خلافه. وذلك أنه كان ينتقل إلى جميع...,2025-08-24 14:14:54,Cassation 17124 شارع نورا برماوي عمارة رقم 87\...
1,85,86518562,29039359,3,Criminal,11,Criminals,29815330,No,Yes,...,"[{'id': 1, 'name': 'Legal'}]",[],[],41,بدور وحيد عبدالرحمن الشيباني,https://via.placeholder.com/640x480.png/005555...,2star,إن يقال انها كثيرة، ولا واحدة، لان الكثرة انما...,2025-08-24 14:15:02,Criminal 29514 شارع زين المقبل\nالحوية Port Sa...
2,4,44581766,37683312,3,Criminal,11,Criminals,11168611,No,Yes,...,"[{'id': 1, 'name': 'Legal'}]",[],[],19,فهد الخالدي,https://via.placeholder.com/640x480.png/00ddee...,4star,البرهان على خلافه. وذلك أنه كان ينتقل إلى جميع...,2025-08-24 14:14:54,Criminal 17124 شارع نورا برماوي عمارة رقم 87\n...
3,49,24157660,65579015,2,Civil,6,Banking,26919441,No,Yes,...,"[{'id': 3, 'name': 'Criminal'}]",[],[],56,يزن برماوي,https://via.placeholder.com/640x480.png/00ffff...,1star,بهائه، ولم يعرض عنه حتى وافته منيته، وهذا على ...,2025-08-24 14:14:54,Civil 94836 ممر الطفيل هوساوي عمارة رقم 65\nسي...
4,49,24157660,65579015,2,Civil,6,Banking,26919441,No,Yes,...,"[{'id': 3, 'name': 'Criminal'}]",[],[],107,رغيد عزت أوس الحسين,https://via.placeholder.com/640x480.png/00aaee...,5star,الكون والفساد ومنها تتركب الأشياء ذوات الصور ا...,2025-08-24 14:15:02,Civil 94836 ممر الطفيل هوساوي عمارة رقم 65\nسي...


In [21]:
### Text Data Preprocessing
lawyers_with_rates['processed_text'] = lawyers_with_rates['Text'].apply(split_by_language)
lawyers_with_rates.head()

,agency_id,agency_sequential,agency_record,court_id,court_name,court_room_id,court_room_name,estimated_cost,is_active,is_archived,...,agencies,issues,user_id,user_name,user_avatar,rating,review,timestamp,Text,processed_text
0,4,44581766,37683312,1,Cassation,1,Cassation,10884613,No,Yes,...,[],[],19,فهد الخالدي,https://via.placeholder.com/640x480.png/00ddee...,4star,البرهان على خلافه. وذلك أنه كان ينتقل إلى جميع...,2025-08-24 14:14:54,Cassation 17124 شارع نورا برماوي عمارة رقم 87\...,"[شارع, ورا, رماو, عمار, رقم, خرج, بره, على, خل..."
1,85,86518562,29039359,3,Criminal,11,Criminals,29815330,No,Yes,...,[],[],41,بدور وحيد عبدالرحمن الشيباني,https://via.placeholder.com/640x480.png/005555...,2star,إن يقال انها كثيرة، ولا واحدة، لان الكثرة انما...,2025-08-24 14:15:02,Criminal 29514 شارع زين المقبل\nالحوية Port Sa...,"[شارع, زي, مقبل, حو, قال, نه, ثير, احد, ان, كث..."
2,4,44581766,37683312,3,Criminal,11,Criminals,11168611,No,Yes,...,[],[],19,فهد الخالدي,https://via.placeholder.com/640x480.png/00ddee...,4star,البرهان على خلافه. وذلك أنه كان ينتقل إلى جميع...,2025-08-24 14:14:54,Criminal 17124 شارع نورا برماوي عمارة رقم 87\n...,"[شارع, ورا, رماو, عمار, رقم, خرج, بره, على, خل..."
3,49,24157660,65579015,2,Civil,6,Banking,26919441,No,Yes,...,[],[],56,يزن برماوي,https://via.placeholder.com/640x480.png/00ffff...,1star,بهائه، ولم يعرض عنه حتى وافته منيته، وهذا على ...,2025-08-24 14:14:54,Civil 94836 ممر الطفيل هوساوي عمارة رقم 65\nسي...,"[ممر, طفيل, هوساو, عمار, رقم, هات, لم, عرض, عن..."
4,49,24157660,65579015,2,Civil,6,Banking,26919441,No,Yes,...,[],[],107,رغيد عزت أوس الحسين,https://via.placeholder.com/640x480.png/00aaee...,5star,الكون والفساد ومنها تتركب الأشياء ذوات الصور ا...,2025-08-24 14:15:02,Civil 94836 ممر الطفيل هوساوي عمارة رقم 65\nسي...,"[ممر, طفيل, هوساو, عمار, رقم, هات, كون, فساد, ..."


In [22]:
results = compute_similarity(lawyers_with_rates,rates['review'][0])
# print(results.values)

In [23]:

filtered_df = lawyers_pere[lawyers_pere['id'].isin(results.values)]
# filtered_df.head()

In [24]:
# Convert to JSON format
results = {
        "isSuccess": True,
        "lawyers": lawyers_pere.to_dict(orient="records")  # Convert DataFrame to list of dictionaries
    }
results

{'isSuccess': True,
 'lawyers': [{'id': 115,
   'name': 'ميساء المقبل',
   'email': 'schroeder.eveline@example.com',
   'address': '54 شارع يونس الشهري\nالهفوف',
   'birthdate': '1988-12-05',
   'birth_place': 'Boganberg',
   'national_number': None,
   'union_branch': 'Lake Rethamouth',
   'union_number': '98921164',
   'affiliation_date': '1971-05-02',
   'years_of_experience': 38,
   'description': "Hatter, 'I cut some more bread-and-butter--' 'But what did the archbishop find?' The Mouse did not like to show you! A little bright-eyed terrier, you know, and he checked himself suddenly: the.",
   'gender': 'female',
   'credit_cart': '8547905628069017',
   'reservation_profit': 234179,
   'conversation_profit': 65993,
   'phone': '+18327336641',
   'avatar': 'https://via.placeholder.com/640x480.png/009999?text=quisquam',
   'rank': 0.0,
   'rates': [],
   'specializations': [{'id': 6, 'name': 'Commercial'}],
   'agencies': [],
   'issues': [],
   'role': 'lawyer'},
  {'id': 116,
   '